In [1]:
# 0) install BERTScore if needed
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00:00:01m00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 4.8 MB/s eta 0:00:000:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing in

In [2]:
import os
import time
import pandas as pd
import torch
from PIL import Image
from tqdm.auto import tqdm
from transformers import ViltProcessor, ViltForQuestionAnswering
from bert_score import score

2025-05-16 17:12:14.154769: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747415534.382561      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747415534.449752      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# 1) load & truncate your QA dataset
df = pd.read_csv("/kaggle/input/test-data-curation/qa_dataset.csv")
df_small = df

# 2) device setup
device    = "cuda" if torch.cuda.is_available() else "cpu"

# 3) load ViLT‐B32 finetuned for VQA
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model     = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa").to(device).eval()

# 4) inference loop with timing
preds, refs = [], []
image_root  = "/kaggle/input/test-data-dataset/images"

preprocessor_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/136k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/470M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/470M [00:00<?, ?B/s]

In [4]:
start_time = time.time()
for _, row in tqdm(df_small.iterrows(), total=len(df_small), desc="ViLT-B32 Inference"):
    # load & preprocess image
    img_path = os.path.join(image_root, row["path"])
    image    = Image.open(img_path).convert("RGB")
    
    # prepare inputs
    encoding = processor(image, row["question"], return_tensors="pt").to(device)
    
    # forward pass
    with torch.inference_mode():
        outputs = model(**encoding)
        logits  = outputs.logits
        idx     = logits.argmax(-1).item()
        pred    = model.config.id2label[idx]
    
    preds.append(pred)
    refs .append(str(row["answer"]).strip())
end_time = time.time()

ViLT-B32 Inference:   0%|          | 0/5994 [00:00<?, ?it/s]

In [9]:
# 5) compute timing
total_time  = end_time - start_time
avg_time_ms = total_time / len(preds) * 1000

# 6) Exact-Match Accuracy
exact_match = sum(p.lower()==r.lower() for p,r in zip(preds, refs)) / len(preds)

# 7) BERTScore F1 (all preds)
P_all, R_all, F1_all = score(preds, refs, lang="en", rescale_with_baseline=True)
bert_f1_all = F1_all.mean().item()

# 8) One-word compliance & BERTScore on one-word preds
is_one_word  = [((" " not in p) and p!="") for p in preds]
pct_one_word = sum(is_one_word) / len(preds) * 100
preds_1w     = [p for p, ok in zip(preds, is_one_word) if ok]
refs_1w      = [r for r, ok in zip(refs,  is_one_word) if ok]

if preds_1w:
    _, _, F1_1w = score(preds_1w, refs_1w, lang="en", rescale_with_baseline=True)
    bert_f1_1w = F1_1w.mean().item()
else:
    bert_f1_1w = float("nan")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# 9) report
print(f"Total inference time (1000 samples): {total_time:.1f}s")
print(f"Average per sample:                {avg_time_ms:.1f}ms")
print()
print(f"Exact-Match Accuracy:              {exact_match:.2f}")
print(f"BERTScore F1 (all preds):          {bert_f1_all:.2f}")
print()
print(f"% One-Word Predictions:            {pct_one_word:.2f}%")
print(f"BERTScore F1 (one-word only):      {bert_f1_1w:.2f}")

Total inference time (1000 samples): 213.5s
Average per sample:                35.6ms

Exact-Match Accuracy:              0.46
BERTScore F1 (all preds):          0.83

% One-Word Predictions:            98.70%
BERTScore F1 (one-word only):      0.84
